In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from langchain_groq import ChatGroq

llamaChatModel = ChatGroq(model="llama-3.3-70b-versatile")

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
chain.invoke("Anuj Rai")

'Anuj Rai'

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [5]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [6]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)


In [7]:
chain.invoke("Anuj")

'Anujovich'

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
   * operation_a will use RunnablePassthrough.
   * operation_b will use RunnableLambda with the russian_lastname function.

In [8]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [9]:
chain.invoke("Anuj Rai")

{'operation_a': 'Anuj Rai', 'operation_b': 'Anuj Raiovich'}

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | llamaChatModel | output_parser

In [12]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

"One curious fact about Roman Abramovich is that he has a unique and somewhat mysterious past. Before becoming a successful businessman and owner of Chelsea Football Club, Abramovich was a street trader in Moscow, selling anything from plastic ducks to rubber toys. However, what's even more fascinating is that he also has a background in the oil industry, and his rapid accumulation of wealth in the 1990s has been shrouded in controversy and speculation. It's been reported that he was able to acquire significant stakes in several large Russian oil companies, including Sibneft, through a series of complex and somewhat opaque deals. This led to him becoming one of the wealthiest men in Russia, with an estimated net worth of over $15 billion. Despite his immense wealth, Abramovich remains an enigmatic figure, and his rise to success is still not fully understood."

## Advanced use of Langchain Parallel

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

In [19]:

load_dotenv()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS index
vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."],
    embedding=embeddings
)


In [21]:
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llamaChatModel
    | StrOutputParser()
)

retrieval_chain.invoke("What is dswithbappy?")

'dswithbappy is an entity that focuses on providing content on various topics such as Data Science, AI, ML, DL, CV, NLP, and Python programming, with all content being in English.'